In [ ]:
# USGS to Parquet Example

In [ ]:
import pandas as pd
import geopandas as gpd
import geoviews as gv
from pathlib import Path
import hvplot.pandas #noqa
import cartopy.crs as ccrs
from holoviews import opts

In [ ]:
# Set some configurations
NGEN_DIR = "/home/jovyan/cache/AWI_03W_113060_001_NGEN/"
NGEN_CONFIG_DIR = Path(NGEN_DIR, "config")
NGEN_FORCINGS_DIR = Path(NGEN_DIR, "forcings")
NGEN_OUTPUT_DIR = Path(NGEN_DIR, "output")

STUDY_DIR = "/home/jovyan/shared-readwrite/rti-eval/ngen-simulation-example/"
STUDY_TS_DIR = Path(STUDY_DIR, "timeseries")
STUDY_GEO_DIR = Path(STUDY_DIR, "geo")
# STUDY_USGS_DIR = Path(STUDY_DIR, "timeseries", "usgs")

In [ ]:
## Create USGS Gage Geometry and Crosswalk

In [ ]:
usgs_gdf = gpd.read_file(Path(STUDY_GEO_DIR, "test_simulation_gages.geojson"))
usgs_gdf.rename(columns={"STAID":"id","STANAME":"name"}, inplace=True, errors="raise")
usgs_gdf["id"] = "usgs-" + usgs_gdf["id"].astype(str)
usgs_gdf = usgs_gdf[["id", "name", "geometry"]]
usgs_gdf.to_parquet(Path(STUDY_GEO_DIR, "usgs_geometry.parquet"))
usgs_gdf

In [ ]:
usgs_points = usgs_gdf.to_crs("EPSG:3857").hvplot(color="orange", size=100, crs=ccrs.GOOGLE_MERCATOR, hover_cols=["id"])

In [ ]:
tiles = gv.tile_sources.OSM

In [ ]:
nexus_file_gdf = gpd.read_file(Path(NGEN_CONFIG_DIR, "nexus_data.geojson")).to_crs("EPSG:3857")
nexus_points = nexus_file_gdf.hvplot(color="green", crs=ccrs.GOOGLE_MERCATOR)

In [ ]:
(tiles * usgs_points * nexus_points).opts(width=800, height=800)

In [ ]:
# This list was generated by hand because the hydrofabric for this example 
# seems to be messed up. In the future, this could be automated based on the hydrofabric data.
nex_location_ids = [
    "nex-114937",
    "nex-114978",
    "nex-113046",
    "nex-113052",
    "nex-113730",
    "nex-114091",
    "nex-113057",
    "nex-114272"
]
usgs_nex_crosswalk = pd.DataFrame({"primary_location_id": usgs_gdf["id"], "secondary_location_id": nex_location_ids})
usgs_nex_crosswalk.to_parquet(Path(STUDY_GEO_DIR, "usgs_nex_crosswalk.parquet"))